In [9]:
import pandas as pd
import torch
import os
import argparse
from tqdm import tqdm

import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.tensorboard import SummaryWriter
from datasets import *
from utils import *
from models import BackBone, ClassifierHead1, ClassifierHead2, ClassifierHead3
from loss_fn import FocalLoss

In [16]:
from predictor import Predictor
import argparse
from easydict import EasyDict

In [17]:
args = EasyDict({"BATCH_SIZE" : 32,
"MODEL_NAME":'efficientnetv2_rw_m',
"MODEL_TYPE" : 'cnn',
"RESIZE" : 224,
"ENSEMBLE" : None,
"IMG_PATH" : "./data/image/test/*",
"CSV_PATH" : "data/train.csv",
"SUB_CSV_PATH" : "./data/sample_submission.csv",
"CLS_CONFIG" : './config/cls_config.yaml',
"LOSS_CONFIG" : './config/loss_config.yaml',
"OUTPUT" : './submission/29E-lr0.01_fullData_maskingWeight_efficientnetv2_m.csv',
"CHECKPOINT" : ['./ckpt/lr0.01_fullData_maskingWeight_efficientnetv2_m/29E-val0.59177165013744-efficientnetv2_rw_m.pth']})

In [18]:
cls_config = read_cfg(args.CLS_CONFIG)
loss_config = read_cfg(args.LOSS_CONFIG)
label_decode = {v: k for k, v in LABEL_ENCODER(args.CSV_PATH).cat3_label_encoder().items()}

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

backbone = BackBone(model_name=args.MODEL_NAME, model_type=args.MODEL_TYPE).to('cuda')
head1 = ClassifierHead1(**cls_config['head1']).to("cuda")
head2 = ClassifierHead2(**cls_config['head2']).to("cuda")
head3 = ClassifierHead3(**cls_config['head3']).to("cuda")

predictor = Predictor(backbone, head1, head2, head3, device, args)

preds = predictor.run()

C:\Users\quhb2\Documents\git\dacon_tourism\utils.py:145: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cat3 = df[df['cat1'] == n_cat1][df['cat2'] == n_cat2]['cat3'].unique()


MODEL LOAD ... from ./ckpt/lr0.01_fullData_maskingWeight_efficientnetv2_m/29E-val0.59177165013744-efficientnetv2_rw_m.pth


C:\Users\quhb2\Documents\git\dacon_tourism\utils.py:145: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cat3 = df[df['cat1'] == n_cat1][df['cat2'] == n_cat2]['cat3'].unique()
100%|████████████████████████████████████████████████████████████████████████████████| 228/228 [01:27<00:00,  2.61it/s]


In [20]:
len(preds)

7280

In [22]:
z = [label_decode[p] for p in preds]
len(z)

7280

In [25]:
predictor.df['cat3'] = z

In [28]:
predictor.df.to_csv(args.OUTPUT, index=False)

In [31]:
for i in range(len(predictor.df['cat3'])) :
    print(predictor.df['cat3'][i])

한식
펜션
한식
도서관
산
한식
바/까페
한식
유적지/사적지
전문상가
박물관
한식
농.산.어촌 체험
사찰
한식
서양식
상설시장
해수욕장
계곡
박물관
전시관
상설시장
야영장,오토캠핑장
해수욕장
한식
유적지/사적지
사찰
이색거리
자연생태관광지
박물관
산
자연휴양림
야영장,오토캠핑장
한식
농.산.어촌 체험
자연생태관광지
박물관
전문상가
농.산.어촌 체험
박물관
산
해수욕장
한식
야영장,오토캠핑장
야영장,오토캠핑장
이색거리
호수
한식
한식
한옥스테이
펜션
한식
한식
트래킹
미술관/화랑
야영장,오토캠핑장
상설시장
바/까페
유적지/사적지
유적지/사적지
박물관
바/까페
모텔
자연휴양림
한식
바/까페
산
유적지/사적지
사찰
한식
한옥스테이
민물낚시
계곡
유적지/사적지
야영장,오토캠핑장
야영장,오토캠핑장
야영장,오토캠핑장
농.산.어촌 체험
전문상가
야영장,오토캠핑장
골프
유적지/사적지
한식
한식
일반축제
야영장,오토캠핑장
강
한식
자연휴양림
한식
사찰
한식
이색거리
유적지/사적지
미술관/화랑
한식
야영장,오토캠핑장
산
일반축제
한식
한식
유적지/사적지
한식
해수욕장
바/까페
한식
사찰
바/까페
한식
박물관
이색거리
한식
공원
바/까페
바/까페
한식
희귀동.식물
유적지/사적지
박물관
이색거리
도서관
야영장,오토캠핑장
사찰
바/까페
유명건물
박물관
박물관
한식
한식
한식
골프
유적지/사적지
모텔
일반축제
야영장,오토캠핑장
한식
상설시장
한식
일반축제
해수욕장
한식
바/까페
한식
박물관
모텔
유적지/사적지
유적지/사적지
산
공원
전시관
한식
상설시장
한식
한식
테마공원
상설시장
유적지/사적지
한식
박물관
서양식
유적지/사적지
박물관
야영장,오토캠핑장
공원
모텔
일반축제
바/까페
해수욕장
야영장,오토캠핑장
유적지/사적지
유적지/사적지
유명건물
야영장,오토캠핑장
산
박물관
전문상가
성
한식
한식
기념탑/기념비/전망대
바/까페
농.산.어촌 체험
한식
유명건물
농.산.어촌 체험
한식
한식
유적지/사적지
한식
일반축제
사찰
이색거리
산
야영장,오토캠핑장
바/까페
이색거리
한식
야영장,오토캠

In [33]:
a = pd.read_csv('./submission/59E-val0.7574-lr0.01_fullData_maskingWeight_efficientnetv2_m.csv')
a

,id,cat3
0,TEST_00000,한식
1,TEST_00001,펜션
2,TEST_00002,한식
3,TEST_00003,도서관
4,TEST_00004,바다낚시
...,...,...
7275,TEST_07275,한식
7276,TEST_07276,이색거리
7277,TEST_07277,"야영장,오토캠핑장"
7278,TEST_07278,수련시설
